In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'weather_data/cities.csv'

In [3]:
city_data_df.dtypes

NameError: name 'city_data_df' is not defined

In [13]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [14]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [17]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 80


In [18]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Atuona,PF,2022-01-31 23:57:25,-9.8000,-139.0333,78.46,77,14,18.12
2,2,Bluff,NZ,2022-01-31 23:57:38,-46.6000,168.3333,75.79,80,37,3.31
6,6,Rikitea,PF,2022-01-31 23:57:40,-23.1203,-134.9692,78.01,67,66,4.72
22,22,Bubaque,GW,2022-01-31 23:57:48,11.2833,-15.8333,77.52,67,88,13.18
27,27,Georgetown,MY,2022-01-31 23:55:06,5.4112,100.3354,78.89,88,20,3.00
32,32,Vao,NC,2022-01-31 23:57:52,-22.6667,167.4833,77.88,84,100,9.06
41,41,Morondava,MG,2022-01-31 23:57:56,-20.2833,44.2833,78.12,79,2,4.12
52,52,Guanica,PR,2022-01-31 23:59:02,17.9716,-66.9080,79.92,77,34,1.01
62,62,The Valley,AI,2022-01-31 23:59:06,18.2170,-63.0578,78.94,73,20,8.05
69,69,Kapaa,US,2022-01-31 23:59:08,22.0752,-159.3190,76.12,59,100,1.01


In [19]:
preferred_cities_df.count()

City_ID       88
City          88
Country       87
Date          88
Lat           88
Lng           88
Max Temp      88
Humidity      88
Cloudiness    88
Wind Speed    88
dtype: int64

In [20]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Atuona,PF,78.46,-9.8000,-139.0333,
2,Bluff,NZ,75.79,-46.6000,168.3333,
6,Rikitea,PF,78.01,-23.1203,-134.9692,
22,Bubaque,GW,77.52,11.2833,-15.8333,
27,Georgetown,MY,78.89,5.4112,100.3354,
32,Vao,NC,77.88,-22.6667,167.4833,
41,Morondava,MG,78.12,-20.2833,44.2833,
52,Guanica,PR,79.92,17.9716,-66.9080,
62,The Valley,AI,78.94,18.2170,-63.0578,
69,Kapaa,US,76.12,22.0752,-159.3190,


In [21]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
   # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [22]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Atuona,PF,78.46,-9.8000,-139.0333,Villa Enata
2,Bluff,NZ,75.79,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
6,Rikitea,PF,78.01,-23.1203,-134.9692,People ThankYou
22,Bubaque,GW,77.52,11.2833,-15.8333,"Casa Dora, Bubaque"
27,Georgetown,MY,78.89,5.4112,100.3354,Cititel Penang
...,...,...,...,...,...,...
548,Nova Russas,BR,79.18,-4.7067,-40.5631,palace motel
558,Cabo San Lucas,MX,76.60,22.8909,-109.9124,Hotel Tesoro Los Cabos
560,Padang,ID,77.05,-0.9492,100.3543,Mercure Padang
561,Soyo,AO,78.39,-6.1349,12.3689,Hotel Mpampa


In [23]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [24]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [25]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{column1}</dd>
<dt>Another name</dt><dd>{column2}</dd>
</dl>
"""

In [26]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [27]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [28]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))